In [1]:
import faiss
from langchain_community.vectorstores import FAISS

In [2]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.vectorstores import FAISS

In [3]:
from langchain.chains import RetrievalQA
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

In [4]:
embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
new_vector_store = FAISS.load_local(
    "../hypnoq_index", embed_model, allow_dangerous_deserialization=True
)

In [6]:
new_vector_store.docstore._dict

{'b76e60a1-3332-436d-ad4c-08aff5c83cad': Document(metadata={'Header 1': 'Frequently Asked Questions about Daydream Therapy (FAQs)', 'Header 2': 'FAQ', 'Header 4': 'What is hypnosis and is it safe?'}, page_content='What is hypnosis and is it safe?\nDuring hypnosis I use this state within you to make positive suggestions to your unconscious mind, based on what you tell me you want to achieve. Your unconscious is in a much more receptive position when you are in this relaxed state. Therefore, it is an incredibly effective tool, to take you towards your solution. When combined with other tools and techniques, such as NLP, can make short work of long-standing problems.  \nAnd yes, it is a safe, and natural state that we all go in and out of many times during our day, perhaps without even knowing it. If you have ever been driving along a road and then ‘come to’ and wondered who was in control of the car for the last five minutes, or been ’miles away’ when someone is trying to talk to you – t

In [23]:
from langchain_groq import ChatGroq # Import Groq's LLM interface (replace with the actual import if it differs)


# Initialize the Groq LLM
groq_llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [48]:
review_template = """Your task is to use the provided content to answer questions about hypnotherapy services offered by the hypnotherapist. 
Use the following context to provide detailed and accurate information. 
Do not create or assume information beyond what is given in the context. 
If the answer is not covered by the provided context, 
kindly encourage the client to reach out to the hypnotherapist directly via email or phone for more information.
{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["context"], template=review_template)
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=groq_llm,
    chain_type="stuff",
    retriever=new_vector_store.as_retriever(search_type="similarity_score_threshold", 
                                            search_kwargs={"score_threshold": .5, 
                                            "k": 3}),
)
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt

In [49]:
# Example question
question = "How effective is hypnotherapy for quitting smoking?"

# Get the answer
answer = reviews_vector_chain.invoke(question)

# Print the answer
print("Answer:", answer['result'])

Answer: Based on the information provided, hypnotherapy can be an effective solution for quitting smoking, but its success largely depends on the individual's willingness to change and commitment to the process. The text mentions that cognitive hypnotherapy is a form of mental reprogramming, which can help individuals learn new ways of thinking, behaving, and feeling.

The hypnotherapist has observed that clients who are curious, open to learning about themselves, and prepared to put in the effort are more likely to achieve positive changes in their lives. This suggests that individuals seeking hypnotherapy for quitting smoking would benefit from having a similar mindset – being open to the process, willing to learn about their habits and triggers, and committed to making a change.

However, it's essential to remember that hypnotherapy is not a "miracle pill" and requires active participation from the client. The hypnotherapist can guide and support the individual throughout the proces